In [1]:
from pathlib import Path
from book_loader import BookLoader
from typing import Dict, List
import pandas as pd

In [2]:
data_path = Path("data/D5627-Dolan.docx").expanduser().resolve()
assert data_path.exists()

In [3]:
book = BookLoader(data_path)
chapters: Dict[int, List[str]] = book.chapters

/home/ahmedm/anaconda3/envs/LDS/lib/python3.9/site-packages/simplify_docx/iterators/generic.py:193: UnexpectedElementWarning: Skipping unexpected tag: {http://schemas.openxmlformats.org/wordprocessingml/2006/main}pict
  warn("Skipping unexpected tag: %s" % (current.tag),


In [4]:
chapters.keys()

dict_keys([0, 1, 2, 3, 4, 5])

In [5]:
print('\n'.join(chapters[0]))

Nous croyons que cette nouvelle édition paraît à point nommé, car le monde organisationnel est soumis à des exigences de productivité historiques, exigences qui demeurent, faut-il l'admettre, complexes à conjuguer avec une promotion et un soutien de la santé psychologique au travail. Si certains cadres envisagent stratégiquement d'accroître la productivité qualitative et quantitative de leur personnel en les poussant à la limite de leurs capacités, trop souvent ces mêmes gestionnaires ont de la difficulté à entrevoir les effets humains collatéraux de cette quête productiviste. Ils ne voient pas (ou simplement, sous-estiment) l'incidence directe qu'a sur leur propre vie et leur santé les pressions psychologiques qu'ils subissent ainsi que celles qu'ils font conséquemment subir à leur équipe de travail. Il serait évidemment utopique d'espérer aujourd'hui rencontrer des contextes de travail exempts de toutes formes de stress. Le père du stress, Hans Selye, l'exprimait assez simplement lor

In [6]:
chapters[0]

["Nous croyons que cette nouvelle édition paraît à point nommé, car le monde organisationnel est soumis à des exigences de productivité historiques, exigences qui demeurent, faut-il l'admettre, complexes à conjuguer avec une promotion et un soutien de la santé psychologique au travail. Si certains cadres envisagent stratégiquement d'accroître la productivité qualitative et quantitative de leur personnel en les poussant à la limite de leurs capacités, trop souvent ces mêmes gestionnaires ont de la difficulté à entrevoir les effets humains collatéraux de cette quête productiviste. Ils ne voient pas (ou simplement, sous-estiment) l'incidence directe qu'a sur leur propre vie et leur santé les pressions psychologiques qu'ils subissent ainsi que celles qu'ils font conséquemment subir à leur équipe de travail. Il serait évidemment utopique d'espérer aujourd'hui rencontrer des contextes de travail exempts de toutes formes de stress. Le père du stress, Hans Selye, l'exprimait assez simplement l

In [7]:
chapters_without_tables = {
    i: list(filter(lambda p: isinstance(p, str), chap))
    for i, chap in chapters.items()}
# chapters_without_tables = map(lambda chap: filter(lambda p: isinstance(p, str), chap),
#                               chapters)
chapters_without_tables[0]

["Nous croyons que cette nouvelle édition paraît à point nommé, car le monde organisationnel est soumis à des exigences de productivité historiques, exigences qui demeurent, faut-il l'admettre, complexes à conjuguer avec une promotion et un soutien de la santé psychologique au travail. Si certains cadres envisagent stratégiquement d'accroître la productivité qualitative et quantitative de leur personnel en les poussant à la limite de leurs capacités, trop souvent ces mêmes gestionnaires ont de la difficulté à entrevoir les effets humains collatéraux de cette quête productiviste. Ils ne voient pas (ou simplement, sous-estiment) l'incidence directe qu'a sur leur propre vie et leur santé les pressions psychologiques qu'ils subissent ainsi que celles qu'ils font conséquemment subir à leur équipe de travail. Il serait évidemment utopique d'espérer aujourd'hui rencontrer des contextes de travail exempts de toutes formes de stress. Le père du stress, Hans Selye, l'exprimait assez simplement l

In [8]:
# Number of words per chapter
df = pd.DataFrame(data={"chapters": chapters.keys()})

df["word_count"] = df["chapters"].apply(
    lambda i: sum(map(lambda p: len(p.split()) if isinstance(p, str) else 0, chapters[i])))

df["unique_word_count"] = df["chapters"].apply(
    lambda i: len(set('\n'.join(chapters[i]).split())))

df.style.hide(axis='index')

TypeError: sequence item 136: expected str instance, list found